## Section 1: Operator Overview 
* Using the most recent month

In [7]:
import _report_utils
import _section1_utils as section1
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis.sql import to_snakecase
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
name = "Bay Area 511 AC Transit Schedule"
organization_name = "Alameda-Contra Costa Transit District"

In [5]:
selected_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = selected_date.year
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

### Map

In [8]:
ac_transit_map = section1.load_operator_map(name)

In [9]:
ac_transit_map.shape

(1593, 14)

In [10]:
ac_transit_map.columns

Index(['shape_array_key', 'geometry', 'feed_key', 'schedule_gtfs_dataset_key',
       'direction_id', 'route_key', 'route_length', 'route_length_miles',
       'organization_source_record_id', 'organization_name', 'service_date',
       'name', 'route_id', 'route_combined_name'],
      dtype='object')

### Operator Profiles

In [25]:
ac_transit_profile = section1.load_operator_profiles(organization_name)

In [26]:
ac_transit_profile

,schedule_gtfs_dataset_key,operator_n_routes,operator_n_trips,operator_n_shapes,operator_n_stops,operator_n_arrivals,operator_route_length_miles,operator_arrivals_per_stop,n_coverage_routes,n_downtown_local_routes,n_local_routes,n_rapid_routes,name,organization_source_record_id,organization_name,service_date
12,c499f905e33929a641f083dad55c521e,132,5478,346,4733,234582,1459.66,49.56,9,112,1,19,Bay Area 511 AC Transit Schedule,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,2024-03-13


In [27]:
ac_transit_profile.name.values[0]

'Bay Area 511 AC Transit Schedule'